## Implementing Chris Doenlen's 'Bot Or Not' Python Module

Everything from `twitter_funcs.py` was cloned from Chris' [repository](https://github.com/scrapfishies/twitter-bot-detection).

I will use this to label each user as 'bot' (boolean 1/0).

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from twitter_funcs import *

# Imports
import os
import numpy as np
import pandas as pd

import pickle
import tweepy

from datetime import datetime
from secrets import api_secret_key, api_key, bearer_token
import re
import time
import csv

In [2]:
data = pd.read_pickle("pickle/n2_tokenized.pick")

In [3]:
data.head(3)

,trump,biden,hashtags,user_id,original,tweet
181142,False,True,[],2820503362,All these articles showing that Biden is in th...,showing joe_biden lead ignore still vote showi...
0,False,True,[],1312487180258820096,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,lady nobody figure life even see nobody help j...
4,False,True,[],2335763630,@The_Grupp “It is purely a fortuity that this ...,purely fortuity great mass casualty history jo...


I'm getting a rate limit error. according to twitter site, I can lookup 300 users per 15 minutes. Let's try that.

In [4]:
def chunks(user_ids, n):
    """Yield successive n-sized chunks from user_ids (iterable)."""
    lst = list(user_ids)
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    


Here's where we'll implement Chris Doenlen's 'Bot or Not' model.

In [5]:
with open("bot_model.pick", "rb") as read_file:
    xgb_model = pickle.load(read_file)

In [6]:
# time.sleep(60*15)

In [ ]:
verifieds = {}
bot_probas = {}

all_users = list(data.user_id.unique())
print(f"Total number of users to scrape: {len(all_users)}")
exist = pd.read_csv("../data/user_stats.csv")
exist_users = list(exist.user_id.unique())
print(f"Number of users already scraped: {len(exist_users)}")

user_ids = []
for user in exist_users:
    if user in all_users:
        continue
    user_ids.append(user)
    
print(f"Preparing to identify bots for {len(user_ids)} users...")

user_id_chunks = list(chunks(user_ids, n=300))

# now get stats for new users

csv_file = open("../data/user_stats.csv", "a")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["user_id", "bot_proba", "verified"])
for chunk in user_id_chunks:
    print(f"Preparing chunk. Num users: {len(chunk)}")
    for user_id in chunk:
        print(f"Preparing user '{user_id}'")
        
        
        auth = tweepy.OAuthHandler(api_key, api_secret_key)
        api = tweepy.API(auth)
        
        try: # Gather features for bot/not bot model
            # Get user information from screen name
            user = api.get_user(user_id)

            # account features to return for predicton
            account_age_days = (datetime.now() - user.created_at).days
            verified = user.verified # will also use this in our data
            geo_enabled = user.geo_enabled
            default_profile = user.default_profile
            default_profile_image = user.default_profile_image
            favourites_count = user.favourites_count
            followers_count = user.followers_count
            friends_count = user.friends_count
            statuses_count = user.statuses_count
            average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

            # manufactured features
            hour_created = int(user.created_at.strftime("%H"))
            network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
            tweet_to_followers = np.round(
                np.log(1 + statuses_count) * np.log(1 + followers_count), 3
            )
            follower_acq_rate = np.round(
                np.log(1 + (followers_count / account_age_days)), 3
            )
            friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

            # organizing list to be returned
            account_features = [
                verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
                followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
                follower_acq_rate,friends_acq_rate]

            if account_features == np.nan:
                proba = np.nan
                verified = np.nan
                csv_writer.writerow([user_id, proba, verified])
                continue

            else:
                user_m = np.matrix(account_features)
                proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
                verified = account_features[0]
                csv_writer.writerow([user_id, proba, verified])

        except:
            print(f'error encountered, skipping user {user_id}')
            proba = np.nan
            verified = np.nan
        
            csv_writer.writerow([user_id, proba, verified])
    print("Chunk complete. Waiting 15 minutes.")
    time.sleep(15*60+1)

csv_file.close()

Total number of users to scrape: 34383
Number of users already scraped: 20101
Preparing to identify bots for 13541 users...
Preparing chunk. Num users: 300
Preparing user '105937456'
Preparing user '46538055'
Preparing user '100625142'
Preparing user '1288981025364860928'
Preparing user '65067002'
Preparing user '2562614485'
Preparing user '2246207654'
Preparing user '1066144442241953792'
Preparing user '1007960516'
Preparing user '1118489465008336897'
Preparing user '1206954619789856769'
Preparing user '1243870145229008896'
Preparing user '1658455681'
Preparing user '989298900'
Preparing user '1267800950321876994'
Preparing user '807989050972000256'
Preparing user '838183322'
Preparing user '893684694759362561'
Preparing user '1282581117632000000'
Preparing user '709149862227484672'
Preparing user '3071943960'
Preparing user '761111078201810944'
Preparing user '757727571807051776'
Preparing user '1194715921908023296'
Preparing user '143975787'
Preparing user '2952274270'
Preparing use

Preparing user '1133485857841848320'
Preparing user '1171412312999288832'
Preparing user '2787215427'
Preparing user '500289229'
Preparing user '2213540755'
Preparing user '1840457510'
Preparing user '824171096'
Preparing user '821029492030377984'
Preparing user '3158618551'
Preparing user '898218643888197632'
Preparing user '97413558'
Preparing user '59810886'
Preparing user '2986724116'
Preparing user '1201176493'
Preparing user '812919639894331392'
Preparing user '1229203145676320769'
Preparing user '74333189'
Preparing user '16843499'
Preparing user '2380623400'
Preparing user '1257544806185799680'
Preparing user '1253818448410316802'
Preparing user '1290427170872066053'
Preparing user '1201662807860756480'
Preparing user '758569471351070720'
Preparing user '277524205'
Preparing user '1247962815035510784'
Preparing user '44748304'
Preparing user '3084606262'
Preparing user '1274357130427056129'
Preparing user '16458709'
Preparing user '195645207'
Preparing user '615915855'
Preparin

Preparing user '2244706798'
Preparing user '3177009068'
Preparing user '249721328'
Preparing user '2772901864'
Preparing user '2936945615'
Preparing user '992188848770822145'
Preparing user '2617596283'
Preparing user '25790437'
Preparing user '1254895741690163200'
Preparing user '415142756'
Preparing user '1335059035'
Preparing user '2259260936'
Preparing user '978307014605639680'
Preparing user '1460958716'
Preparing user '1548506839'
Preparing user '1323229114111844353'
Preparing user '709103640062066688'
Preparing user '1300177071071690753'
Preparing user '4319693115'
Preparing user '2291120303'
Preparing user '3991537240'
Preparing user '139596582'
Preparing user '1262830448176234497'
Preparing user '30500949'
Preparing user '1296152764280709121'
Preparing user '31169435'
Preparing user '2863437856'
Preparing user '1061410712831045632'
Preparing user '1311862962903298049'
Preparing user '1337922428'
Preparing user '49478496'
Preparing user '1087520123089575937'
Preparing user '979

Preparing user '19604028'
Preparing user '1049474297134223360'
Preparing user '2722275065'
Preparing user '26042009'
Preparing user '1217173636634808321'
Preparing user '1218032038386769920'
Preparing user '713461348034813956'
Preparing user '91981041'
Preparing user '115071051'
Preparing user '122254934'
Preparing user '1299152836182245376'
Preparing user '3926587454'
Preparing user '862154071536312320'
Preparing user '1070096619314524162'
Preparing user '125502420'
Preparing user '1283547777948160001'
Preparing user '2361817734'
Preparing user '944758932442157056'
Preparing user '1597550012'
Preparing user '559718461'
Preparing user '1004491234683994112'
Preparing user '1239351932550107136'
Preparing user '1046130110'
Preparing user '1852601298'
Preparing user '2163022478'
Preparing user '1293531609933721600'
Preparing user '133088013'
Preparing user '1403361253'
Preparing user '2574582585'
Preparing user '747854636476948481'
Preparing user '120839423'
Preparing user '344453560'
Prep

Preparing user '834030626185572353'
Preparing user '866749606754910208'
Preparing user '1315050522399903746'
Preparing user '1313850516208840704'
Preparing user '924776360'
Preparing user '970894987264217088'
Preparing user '3503381716'
Preparing user '1072572644036358146'
Preparing user '1299863840432160769'
Preparing user '10221'
Preparing user '1140049316184317953'
Preparing user '1197284111007830016'
Preparing user '43429285'
Preparing user '936810972551024640'
Preparing user '194681383'
Preparing user '1030301449547210752'
Preparing user '1314641883235258373'
Preparing user '773451838939099137'
Preparing user '3369255861'
Preparing user '31326363'
Preparing user '2634695718'
Preparing user '863982793319215106'
Preparing user '801362359877009408'
Preparing user '163657957'
Preparing user '1115505571908804610'
Preparing user '1182070033587036162'
Preparing user '874339847439355907'
Preparing user '1289418249222807552'
Preparing user '1320899712103440387'
Preparing user '889033997942

...

Preparing user '1014218457506811904'
Preparing user '4047080903'
Preparing user '1463381107'
Preparing user '33112847'
Preparing user '4064163214'
Preparing user '2350393730'
Preparing user '378530969'
Preparing user '285428167'
Preparing user '1300910335'
Preparing user '1306689032722751503'
Preparing user '1281694075918442497'
Preparing user '881012873862033409'
Preparing user '788524'
Preparing user '1076986333'
Preparing user '2318222995'
Preparing user '1292605075'
Preparing user '27164756'
Preparing user '1322204248017301505'
Preparing user '3401129937'
Preparing user '1230289514876325891'
Preparing user '916527992238731264'
Preparing user '726003942597890048'
Preparing user '948740944614100992'
Preparing user '1314374430513004544'
Preparing user '1019325512425070592'
Preparing user '1112725572046610432'
Preparing user '1158120718288207873'
Preparing user '914915437728686080'
Preparing user '2554837140'
Preparing user '17228464'
Preparing user '1911989682'
Preparing user '9594283

Preparing user '284052760'
Preparing user '866607267990220800'
Preparing user '1642402514'
Preparing user '237986911'
Preparing user '4485994042'
Preparing user '3517880953'
Chunk complete. Waiting 15 minutes.
Preparing chunk. Num users: 300
Preparing user '1195348415431028738'
Preparing user '4909107696'
Preparing user '234974960'
Preparing user '347661208'
Preparing user '1296666174936113154'
Preparing user '3044828098'
Preparing user '330613684'
Preparing user '1028031355639218176'
Preparing user '1615454654'
Preparing user '705994443153920000'
Preparing user '1702348014'
Preparing user '483517190'
Preparing user '14171597'
Preparing user '24728591'
Preparing user '1773774002'
Preparing user '1042569900831789056'
Preparing user '1026991410573062144'
Preparing user '1248291297758232577'
Preparing user '18452400'
Preparing user '825517357695131649'
Preparing user '3053341664'
Preparing user '115149344'
Preparing user '1319661035368910849'
Preparing user '620690358'
Preparing user '399

Preparing user '4159909514'
Preparing user '1313188005385912320'
Preparing user '788155728443957249'
Preparing user '42366930'
Preparing user '2723504769'
Preparing user '65390037'
Preparing user '52638071'
Preparing user '1022846346506764288'
Preparing user '1265754836349472769'
Preparing user '1267105586040143874'
Preparing user '818271396010655745'
Preparing user '1277296390683725830'
Preparing user '229501164'
Preparing user '1235345590290182144'
Preparing user '540489050'
Preparing user '762328387'
Preparing user '195919468'
Preparing user '1018529353104175105'
Preparing user '1214026995060723712'
Preparing user '1319574196847104000'
Preparing user '3529462040'
Preparing user '87079705'
Preparing user '33451457'
Preparing user '118570392'
Preparing user '1030752618723901440'
Preparing user '1163168997984079872'
Preparing user '1304944064572329984'
Preparing user '243046596'
Preparing user '2238402254'
Preparing user '950365820'
Preparing user '1254425441378729985'
Preparing user '

Preparing user '2327404376'
Preparing user '2895313437'
Preparing user '2201552245'
Preparing user '101608330'
Preparing user '4153050136'
Preparing user '19241322'
Preparing user '2339891'
Preparing user '1199464397309009920'
Preparing user '44234992'
Preparing user '197957289'
Preparing user '3050402977'
Preparing user '758151466985066496'
Preparing user '937709427360014336'
Preparing user '986356435691991040'
Preparing user '3284181626'
Preparing user '1053409061734178816'
Preparing user '902448516'
Preparing user '825377278645379072'
Preparing user '16537818'
Preparing user '2732954837'
Preparing user '12000302'
Preparing user '1211417347702964224'
Preparing user '34093053'
Preparing user '1048287884678242304'
Preparing user '1308451745492799494'
Preparing user '1013929214'
Preparing user '2966457893'
Preparing user '198357693'
Preparing user '2808306433'
Preparing user '1266066300645847046'
Preparing user '1191643710'
Preparing user '1232732942524440578'
Preparing user '3548571075

Preparing user '1273172469541675008'
Preparing user '1468348021'
Preparing user '1253519716687081476'
Preparing user '2162628020'
Preparing user '974846751235428353'
Preparing user '979940192919400449'
Preparing user '138551940'
Preparing user '35889525'
Preparing user '787801631580618752'
Preparing user '18780322'
Preparing user '270020383'
Preparing user '21508174'
Preparing user '705077094674522112'
Preparing user '817443159928164353'
Preparing user '1614150008'
Preparing user '905954569'
Preparing user '918161578926792705'
Preparing user '1309598703196549120'
Preparing user '32281804'
Preparing user '703368447690940420'
Preparing user '42805100'
Preparing user '351876611'
Preparing user '1725407809'
Preparing user '1313134239882371079'
Preparing user '1236884148'
Preparing user '20541400'
Preparing user '1050230137478938624'
Preparing user '19986444'
Preparing user '69167813'
Preparing user '33594728'
Preparing user '1205354752499834880'
Preparing user '1073000680997171200'
Prepari

In [ ]:
bot_probas

In [ ]:
data['bot_proba'] = data['user_id'].map(bot_probas)
data['verifieds'] = data['user_id'].map(is_verified)
data.to_pickle("pickle/tw_proba_verif.pick")

In [ ]:
verifieds

In [ ]:
data['verified'] = data['user_id'].map(verifieds)
data['bot_proba'] = data['user_id'].map(bot_probas)
